<a href="https://colab.research.google.com/github/pddmadushan/2024-Hackathon-Team-Ai-Teco/blob/main/Untitled26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [9]:
import openai

# Set your OpenAI API key
openai.api_key = ""  # Replace with your actual API key

# Example usage of the generator
def generate_code(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4o",  # Use "gpt-3.5-turbo" or any other model you prefer
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500,
    )
    return response['choices'][0]['message']['content']

In [12]:
# Step 1: Install Required Libraries
!pip install -q torch transformers sentence-transformers faiss-cpu accelerate pyngrok flask_cors

# Step 2: Load Required Libraries
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Step 3: Load and Parse RAG Text File
file_path = '/content/SampleTestCode2.txt'  # Change this to your actual file path

with open(file_path, 'r') as f:
    text_data = f.read()

# Step 4: Parsing Logic to Extract Chunks Enclosed in `### ... ###`
import re

# Pattern to match any text enclosed by `### ... ###`
pattern = r'###(.*?)###'

# Extract all matches for text enclosed by ###
matches = re.findall(pattern, text_data, re.DOTALL)

# Check if matches are detected and display them for debugging
if not matches:
    print("No matches found. Please check the file format.")
else:
    print(f"Total matches found: {len(matches)}")

# Format chunks by stripping any extra whitespace
formatted_chunks = [match.strip() for match in matches]

# Final check on chunks
print(f"Number of formatted chunks: {len(formatted_chunks)}")
if formatted_chunks:
    print(f"First formatted chunk:\n{formatted_chunks[0]}")

# Step 5: Load Sentence Transformer Model for Embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 6: Create Embeddings for Each Chunk
embeddings = embedding_model.encode(formatted_chunks)

# Check the shape of embeddings
print(f"Embeddings shape: {embeddings.shape}")

# Ensure embeddings are valid before proceeding
if embeddings.size == 0:
    raise ValueError("No embeddings were created. Please check your input data.")

# Step 7: Create a FAISS Index for Efficient Retrieval
dimension = embeddings.shape[1]  # Get the number of dimensions for FAISS index
index = faiss.IndexFlatL2(dimension)  # L2 distance index
index.add(np.array(embeddings))  # Add your embeddings to the index

print("FAISS index created and embeddings added successfully.")

# Step 8: Define Retrieval Function with Deduplication
def retrieve_code(query):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k=1)  # Retrieve only the top 1 similar chunk

    # Get the most relevant chunk based on the top index
    top_chunk = formatted_chunks[indices[0][0]] if indices[0][0] < len(formatted_chunks) else None

    # Clean up any unexpected whitespace or newlines
    return top_chunk.strip() if top_chunk else None

Total matches found: 3
Number of formatted chunks: 3
First formatted chunk:
Prompt: Create a Sales Order
namespace TestsInternal.Tests { // Create the class and Check public class SalesOrderTC : Check { // Create an instance of the OrderSoQP OrderSoQP SalesOrder = new OrderSoQP(); public override void Execute() { // Log in to the destination site PxLogin.LoginToDestinationSite(); #region Testcase1: Create a Sales Order using (TestExecution.CreateTestCaseGroup("Testcase1: Create a Sales Order")) { #region Teststep2: Add a sales order using (TestExecution.CreateTestStepGroup("Teststep2: Add a sales order")) { // Open the Sales Order screen and perform actions SalesOrder.OpenScreen(); SalesOrder.Insert(); SalesOrder.Summary.CustomerID.Select(""); // Add appropriate customer ID SalesOrder.Summary.OrderDesc.Type(""); // Provide the order description SalesOrder.Save(); // Save the order } #endregion } #endregion } } }


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embeddings shape: (3, 384)
FAISS index created and embeddings added successfully.
Retrieved Context:

Prompt: Create a Sales Order
namespace TestsInternal.Tests { // Create the class and Check public class SalesOrderTC : Check { // Create an instance of the OrderSoQP OrderSoQP SalesOrder = new OrderSoQP(); public override void Execute() { // Log in to the destination site PxLogin.LoginToDestinationSite(); #region Testcase1: Create a Sales Order using (TestExecution.CreateTestCaseGroup("Testcase1: Create a Sales Order")) { #region Teststep2: Add a sales order using (TestExecution.CreateTestStepGroup("Teststep2: Add a sales order")) { // Open the Sales Order screen and perform actions SalesOrder.OpenScreen(); SalesOrder.Insert(); SalesOrder.Summary.CustomerID.Select(""); // Add appropriate customer ID SalesOrder.Summary.OrderDesc.Type(""); // Provide the order description SalesOrder.Save(); // Save the order } #endregion } #endregion } } }


In [19]:
# Import necessary modules
from flask import Flask, request, jsonify  # Flask for web server functionality
from pyngrok import ngrok  # pyngrok for creating secure tunnels to local servers
import requests  # requests for making HTTP requests
from flask_cors import CORS  # Import CORS for Cross-Origin Resource Sharing
import cv2  # OpenCV for image processing
import numpy as np  # NumPy for numerical computing
import tensorflow as tf  # TensorFlow for deep learning

In [20]:
port_no = 5000 #Defining Port

In [24]:
# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)

# Set ngrok authentication token
ngrok.set_auth_token("2cOOhCeMaiDYjZWi1b1NGCYfLnO_5FqABvBX85mm9gNZf5jnK")

# Connect to ngrok and get public URL
port_no = 5000  # Set the port number
public_url = ngrok.connect(port_no).public_url

# Define route for home page
@app.route("/")
def home():
    return f"Running Flask on Google Colab!"  # Return a message indicating Flask is running

# Print the public URL
print(f"To access the Global link please click {public_url}")

To access the Global link please click https://249e-35-236-212-91.ngrok-free.app


In [ ]:
# Define route for generating responses
@app.route('/generate_response', methods=['POST'])
def generate_response():
    # Get prompt from the request
    data = request.json
    userQuery = data.get('userQuery', '')

    print(userQuery)

    # Wrap the prompt using the right chat template
    relevant_chunks = retrieve_code(userQuery)
    context = relevant_chunks
    prompt = f"Based on the following code snippets: {context}, please only  the generate code for and dont add comments as well : {userQuery}"

    # Trim the response, remove instruction manually
    response = generate_code(prompt)

    print(response)

    # Return the generated response as JSON
    return jsonify({"response": response})

# Start the Flask app
if __name__ == '__main__':
    app.run(port=port_no)  # Run the app on the specified port


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


create a sales order


INFO:werkzeug:127.0.0.1 - - [25/Oct/2024 15:58:24] "POST /generate_response HTTP/1.1" 200 -


```csharp
namespace TestsInternal.Tests {
    public class SalesOrderTC : Check {
        OrderSoQP SalesOrder = new OrderSoQP();
        public override void Execute() {
            PxLogin.LoginToDestinationSite();
            using (TestExecution.CreateTestCaseGroup("Testcase1: Create a Sales Order")) {
                using (TestExecution.CreateTestStepGroup("Teststep2: Add a sales order")) {
                    SalesOrder.OpenScreen();
                    SalesOrder.Insert();
                    SalesOrder.Summary.CustomerID.Select("");
                    SalesOrder.Summary.OrderDesc.Type("");
                    SalesOrder.Save();
                }
            }
        }
    }
}
```
Create a Journal Entry with file path /content/SampleTestCode2.txt


...

```csharp
namespace TestsInternal.Tests {
    public class SalesOrderTC : Check {
        OrderSoQP SalesOrder = new OrderSoQP();
        public override void Execute() {
            PxLogin.LoginToDestinationSite();
            using (TestExecution.CreateTestCaseGroup("Testcase1: Create a Sales Order")) {
                using (TestExecution.CreateTestStepGroup("Teststep2: Add a sales order")) {
                    SalesOrder.OpenScreen();
                    SalesOrder.Insert();
                    SalesOrder.Summary.CustomerID.Select("Cust123");
                    SalesOrder.Summary.OrderDesc.Type(""); // Provide the order description if needed
                    SalesOrder.Save();
                }
            }
        }
    }
}
```
Create Sales Order using Customer ID Cust123


INFO:werkzeug:127.0.0.1 - - [25/Oct/2024 17:05:42] "POST /generate_response HTTP/1.1" 200 -


Certainly! Here's the code snippet with the specified modifications:

```csharp
namespace TestsInternal.Tests {
    public class SalesOrderTC : Check {
        OrderSoQP SalesOrder = new OrderSoQP();
        public override void Execute() {
            PxLogin.LoginToDestinationSite();
            using (TestExecution.CreateTestCaseGroup("Testcase1: Create a Sales Order")) {
                using (TestExecution.CreateTestStepGroup("Teststep2: Add a sales order")) {
                    SalesOrder.OpenScreen();
                    SalesOrder.Insert();
                    SalesOrder.Summary.CustomerID.Select("Cust123");
                    SalesOrder.Summary.OrderDesc.Type("");
                    SalesOrder.Save();
                }
            }
        }
    }
}
```
create sales order


INFO:werkzeug:127.0.0.1 - - [25/Oct/2024 17:39:36] "POST /generate_response HTTP/1.1" 200 -


Certainly! Here is the code to create a sales order based on the provided snippet:

```csharp
namespace TestsInternal.Tests {
    public class SalesOrderTC : Check {
        OrderSoQP SalesOrder = new OrderSoQP();
        public override void Execute() {
            PxLogin.LoginToDestinationSite();
            using (TestExecution.CreateTestCaseGroup("Testcase1: Create a Sales Order")) {
                using (TestExecution.CreateTestStepGroup("Teststep2: Add a sales order")) {
                    SalesOrder.OpenScreen();
                    SalesOrder.Insert();
                    SalesOrder.Summary.CustomerID.Select("");
                    SalesOrder.Summary.OrderDesc.Type("");
                    SalesOrder.Save();
                }
            }
        }
    }
}
```
